In [3]:
import dask.dataframe as dd
import os
import scipy.stats as stats
import numpy as np
import hvplot.pandas
import pandas as pd

In [4]:
from dask.distributed import Client

In [5]:
client = Client("tcp://127.0.0.1:9500")
client.restart()

Client Scheduler: tcp://127.0.0.1:9500 Dashboard: http://127.0.0.1:9501/status,Cluster Workers: 60 Cores: 180 Memory: 1.62 TB


In [6]:
simulation_data_folder = os.path.join('../tests/data/simulations')
complete_random_folder = os.path.join(simulation_data_folder, 'jcoin_random')
urn_random_folder = os.path.join(simulation_data_folder, 'jcoin_urn')
range_folder = os.path.join(simulation_data_folder, 'd_range')
b2folder = os.path.join(simulation_data_folder, 'b2')
custom_d_folder = os.path.join(simulation_data_folder, 'jcoin_custom_d')

In [12]:
def compute_study_level_trt_assignments(pdf, n_participants=2500):
    pdf_stats = None
    lst_trt = sorted(pdf.trt.unique().tolist())
    for trial_no in pdf.trial_no.unique():
        for n_participants in range(1, n_participants + 1):
            pdf_trial_n = pdf.iloc[:n_participants, ]
            pdf_trial_stats = pdf_trial_n.groupby('trt').size().reset_index(drop=False)
            pdf_trial_stats = pdf_trial_stats.set_index('trt').T
            pdf_trial_stats.columns = ['trt_' + str(col) for col in list(pdf_trial_stats.columns)]
            pdf_trial_stats = pdf_trial_stats.reset_index(drop=True)
            pdf_trial_stats['n_participants'] = n_participants
            pdf_trial_stats['trial_no'] = trial_no
            for trt in lst_trt:
                if 'trt_{0}'.format(trt) not in pdf_trial_stats.columns:
                    pdf_trial_stats['trt_{0}'.format(trt)] = 0
            pdf_trial_stats = pdf_trial_stats[['n_participants', 'trial_no'] + ['trt_{0}'.format(trt) for trt in lst_trt]]
            pdf_stats = pdf_trial_stats.copy() if pdf_stats is None else pd.concat([pdf_stats, pdf_trial_stats], ignore_index=True)
    return pdf_stats

In [13]:
for simulation in [complete_random_folder, urn_random_folder, range_folder,b2folder, custom_d_folder ]:
    print(simulation)
    df_simulation_data = dd.read_csv([os.path.join(simulation, 'trial_{0}.csv'.format(trial_no))
                                      for trial_no in range(1, 1001)])
    df_simulation_data = df_simulation_data.set_index('trial_no', drop=False)
    df_stats = df_simulation_data.map_partitions(lambda pdf: compute_study_level_trt_assignments(pdf, n_participants=2500),
                                                   meta=pd.DataFrame(columns=['n_participants', 'trial_no'] + ['trt_{0}'.format(trt) for trt in sorted(df_simulation_data.head(1000).trt.unique().tolist())]))
    df_stats.to_csv(os.path.join(simulation, 'overall_stats.csv'), index=False, single_file=True)

../tests/data/simulations/jcoin_random
../tests/data/simulations/jcoin_urn
../tests/data/simulations/d_range
../tests/data/simulations/b2
../tests/data/simulations/jcoin_custom_d
